# SKLearn介绍(SciKit-Learn)

#1 模块介绍

##1.1 监督学习
- neighbors:近邻算法
- svm：支持向量机
- kernal_ridge：核岭回归
- discriminant_analysis:判别分析
- linear_model:广义线性模型
- ensemble:集成方法，包括随机森林，bagging
- tree:决策树
- naive_bayes：朴素贝叶斯
- cross_decomposition：交叉分解
- gaussian_process：高斯过程
- neural_network：多层神经网络
- calibration:概率校准
- isotonic：保序回归
- feature_selection：监督特征选择
- multiclass：多类多标签算法

##1.2 无监督学习
- decomposition:矩阵因子分解
- cluster：聚类分析
- manifold：流形学习
- mixture:高斯混合模型
- neural_network:无监督神经网络
- density：密度估计
- covariance：协方差估计

##1.3 数据变换
- preprocessing：数据预处理。包括数据缺失值的处理，数据的规范化，数据的归一化等等。比如数据原来不是标准正太分布
- feature_extraction:特征抽取。只负责把特征提取出来
- feature_selection:特征选择。留下好的特征，丢掉不好的特征
- random_projection:随机投影。将特征进行空间变换,将特征映射到好处理的空间上去
- kernel_approximation：核逼近。将特征进行空间变换,将特征映射到好处理的空间上去
- pipeline:管道流。

#2 六大板块与统一调用API
分类、回归、聚类、维数约简、特征抽取选择、数据预处理
```Python
estimator.fit(x_train,[y_train])
```
- 对于Classification、Regression和Clustering,有predict函数
- 对于Preprocessing、Dimensionality Reduction、Feature Extraction和Feature selection，有transform函数

#3 模型选择与评估

##3.1 数据集划分方法

###3.1.1 K折交叉验证
在机器学习中，将数据集A分为训练集（training set）B和测试集（test set）C，在样本量不充足的情况下，为了充分利用数据集对算法效果进行测试，将数据集A随机分为k个不相交包，每次将其中一个包作为测试集，剩下k-1个包作为训练集进行训练，可以得到k个学习器，然后求出这k次的分类率平均值作为该模型的真实分类率
- KFold
- GroupKFold
- StratifiedKFold

In [5]:
import numpy as np
from sklearn.model_selection import KFold
x = np.array([[1,2],[3,4],[5,6],[7,8],[9,10],[11,12]])
y = np.array([1,2,3,4,5,6])
kf = KFold(n_splits=2)
kf.get_n_splits(x)

for train_index, test_index in kf.split(x):
    print('Train Index:', train_index, ', Test Index:', test_index)
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]

Train Index: [3 4 5] , Test Index: [0 1 2]
Train Index: [0 1 2] , Test Index: [3 4 5]


###3.1.2 留一法
留一法就是每次只留下一个样本做测试集，其它样本做训练集，如果有k个样本，则需要训练k次，测试k次。留一发计算最繁琐，但样本利用率最高。适合于小样本的情况。最有用所有分类器的平均值来衡量模型的性能

如果留下P个样本，那就是留P法
- LeaveOneOut
- LeaveOneGroupOut
- LeavePOut
- LeavePGroupsOut

###3.1.3 随机划分
- ShuffleSplit
- GroupShuffleSplit
- StratifiedShuffleSplit

##3.2 超参数优化方法

###3.2.1 超参数Hyper-Parameters
学习器模型中一般有两类参数
- 模型参数：可以从数据中学习估计得到
- 超参数：无法从数据中估计，只能靠人的经验进行设计指定。比如SVM里面的C、Kernel、$\gamma$，朴素贝叶斯里的$\alpha$等

可以通过```get_params()```来获取学习器的参数和取值

参数空间的搜索包括
- 学习器
- 参数空间
- 用于获得候选参数组合的搜索或采样方法
- 交叉验证机制
- 评分函数

###3.2.2 超参数优化方法
SKLearn提供了两种通用的参数优化方法：网格搜索、随机采样

- 网格搜索交叉验证(GridSearchCV)：以穷举的方式遍历所有可能的参数组合
- 随机采样交叉验证(RandomizedSearchCV)：依据某种分布对参数空间采样，随机的得到一些候选参数组合方案

In [11]:
import numpy as np
from time import time
from scipy.stats import randint as sp_randint
from sklearn import model_selection,datasets,ensemble

def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print('Model with rank: {0}'.format(i))
            print('Mean validation score: {0:.3f} (std: {1:.3f})'.format(results['mean_test_score'][candidate],
                                                                         results['std_test_score'][candidate]))
            print('Parameters:{0}'.format(results['params'][candidate]))
            print("")
digits = datasets.load_digits()
x, y = digits.data, digits.target
clf = ensemble.RandomForestClassifier(n_estimators=20)
print('===============RandomizedSearchCV的结果==============================')
param_dist = {'max_depth':[3, None],
              'max_features':sp_randint(1,11),
              'min_samples_split':sp_randint(2,11),
              'min_samples_leaf':sp_randint(1,11),
              'bootstrap':[True, False],
              'criterion':['gini', 'entropy']}
n_iter_search = 20
random_search = model_selection.RandomizedSearchCV(clf, param_distributions=param_dist, n_iter=n_iter_search)
start = time()
random_search.fit(x, y)
print('RandomizedSearchCV took %.2f seconds for %d candidates parameter settings.' % ((time() - start), n_iter_search))
report(random_search.cv_results_)

print('===============GridSearchCV的结果==============================')
param_grid = {'max_depth':[3, None],
              'max_features':[1,3,10],
              'min_samples_split':[2,3,10],
              'min_samples_leaf':[1,3,10],
              'bootstrap':[True, False],
              'criterion':['gini', 'entropy']}

grid_search = model_selection.GridSearchCV(clf, param_grid=param_grid)
start = time()
grid_search.fit(x, y)
print('GridSearchCV took %.2f seconds for %d candidates parameter settings.' % ((time() - start, len(grid_search.cv_results_['params']))))
report(grid_search.cv_results_)

===============RandomizedSearchCV的结果==============================


RandomizedSearchCV took 6.68 seconds for 20 candidates parameter settings.
Model with rank: 1
Mean validation score: 0.916 (std: 0.018)
Parameters:{'min_samples_leaf': 2, 'criterion': 'gini', 'max_depth': None, 'min_samples_split': 5, 'bootstrap': True, 'max_features': 3}

Model with rank: 2
Mean validation score: 0.914 (std: 0.018)
Parameters:{'min_samples_leaf': 8, 'criterion': 'entropy', 'max_depth': None, 'min_samples_split': 6, 'bootstrap': False, 'max_features': 10}

Model with rank: 3
Mean validation score: 0.914 (std: 0.014)
Parameters:{'min_samples_leaf': 3, 'criterion': 'entropy', 'max_depth': None, 'min_samples_split': 5, 'bootstrap': True, 'max_features': 7}

===============GridSearchCV的结果==============================


GridSearchCV took 58.18 seconds for 216 candidates parameter settings.
Model with rank: 1
Mean validation score: 0.939 (std: 0.004)
Parameters:{'min_samples_leaf': 1, 'criterion': 'gini', 'max_depth': None, 'min_samples_split': 2, 'bootstrap': False, 'max_features': 10}

Model with rank: 2
Mean validation score: 0.936 (std: 0.013)
Parameters:{'min_samples_leaf': 1, 'criterion': 'gini', 'max_depth': None, 'min_samples_split': 3, 'bootstrap': True, 'max_features': 10}

Model with rank: 3
Mean validation score: 0.930 (std: 0.020)
Parameters:{'min_samples_leaf': 3, 'criterion': 'entropy', 'max_depth': None, 'min_samples_split': 2, 'bootstrap': False, 'max_features': 10}



随机搜索的运行时间比网格搜索显著的少，随机搜索得到的超参数组合的性能稍差一些，但这很大程度上时候噪声引起的
优化方法
- 指定一个合适的目标测度对模型进行评估
- 使用SKLearn的PipeLine将estimators和他们的参数空间结合起来
- 合理划分数据集。model_selection.train_test_split()来搞定
- 在参数节点的计算上可以做到并行计算，通过参数n_jobs来指定

##3.3 模型验证方法